In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import tushare as ts
import openpyxl as oxl
import os
from collections import OrderedDict
import datetime
import json


def load_from_local():
    
    with open('__ HKEX __ HKEXnews __.html','r',encoding="utf-8") as f:
        return f.read()

def get_first_part():

    temp_html = soup.find("table",{"id":"Table5"})
    temp_html = temp_html.find_all('td',{"class":"arial12black"})

    first_dict = {}
    temp_string = temp_html[1].get_text().strip()
    temp_string = datetime.datetime.strptime(temp_string,"%d/%m/%Y")
    temp_string = temp_string.strftime("%Y-%m-%d")
    first_dict['last_date'] = temp_string
    first_dict['stock_code'] = temp_html[3].get_text().strip()
    first_dict['stock_name'] = temp_html[5].get_text().strip()

    return first_dict

def get_second_part():

    temp_html = soup.find("div",{"id":{"pnlResultSummary"}})
    temp_html = temp_html.find_all("span",{"class":"mobilezoom"})

    second_dict = {}
    second_dict['hold_volumn'] = temp_html[0].get_text().strip()
    second_dict['people_number'] = temp_html[1].get_text().strip()
    second_dict['hold_precent'] = temp_html[2].get_text().strip()
    second_dict['all_volumn'] = temp_html[6].get_text().strip()

    return second_dict

def get_third_part(first_dict):

    temp_html = soup.find("table",{"id":{"participantShareholdingList"}})
    temp_html = temp_html.find_all('tr')

    row_number = len(temp_html)
    col_number = len(temp_html[0].find_all('td'))

    d = {}
    for i in range(col_number):
        d[i] = []

    for i in range(1,row_number):
        temp_tr = temp_html[i]
        temp_td = temp_tr.find_all('td')
    
        if len(temp_td) !=col_number:
            continue
        
        for j in range(col_number):
            temp_string = temp_td[j].get_text().strip()
            d[j].append(temp_string)

    last_pd = pd.DataFrame(d)
    
    last_dict = {}
    for i in range(len(last_pd)):
        last_dict[last_pd.iat[i,0]] = last_pd.iat[i,1]

    name_dict.update(last_dict)

    new_pd = pd.DataFrame(last_pd[3])
    new_pd = new_pd.T
    new_pd.columns = last_pd[0]

    last_date = first_dict['last_date']
    time_span = pd.Timestamp(last_date)
    new_pd.index = [time_span]

    temp_string = datetime.datetime.strptime(last_date,'%Y-%m-%d')
    end_date = temp_string.date()
    start_date = end_date - datetime.timedelta(10)

    temp_df = ts.get_k_data('601888',start=start_date.isoformat(),end=end_date.isoformat())

    if temp_df.empty:
        #  close_data[last_date] = list(close_data.values())[-1]
        close_data[str(time_span)] = 0 
    else:
        close_data[str(time_span)] = temp_df.iat[-1,2]

    return new_pd

def save_to_excel(first_dict,second_dict,merge_pd):

    wb = oxl.Workbook()
    ws = wb.create_sheet(index=0,title='oxl-sheet')
    
    ws.cell(row=1,column=1).value = '持股日期'
    ws.cell(row=1,column=2).value = first_dict['last_date']
    ws.cell(row=1,column=3).value = '股票代码'
    ws.cell(row=1,column=4).value = first_dict['stock_code']
    ws.cell(row=1,column=5).value = '股票名称'
    ws.cell(row=1,column=6).value = first_dict['stock_name']

    ws.cell(row=3,column=1).value = '中央结算系统持股量'
    ws.cell(row=4,column=1).value = second_dict['hold_volumn']
    ws.cell(row=3,column=2).value = '参与者数目'
    ws.cell(row=4,column=2).value = second_dict['people_number']
    ws.cell(row=3,column=3).value = '总数百分比'
    ws.cell(row=4,column=3).value = second_dict['hold_precent']
    ws.cell(row=3,column=4).value = '全部持股量'
    ws.cell(row=4,column=4).value = second_dict['all_volumn']

    columns = merge_pd.columns
    index = merge_pd.index
    
    row_number = len(index)
    col_number = len(columns)

    ws.cell(row=8,column=1).value = '日期'
    ws.cell(row=8,column=2).value = '收盘价'

    for i in range(row_number):
        ws.cell(row=i+9,column=1).value = str(index[i])[0:10]
        ws.cell(row=i+9,column=2).value = close_data.get(str(index[i]),0)

    for i in range(col_number):
        ws.cell(row=8,column=i+3).value = columns[i]
        ws.cell(row=7,column=i+3).value = name_dict[columns[i]] 

    for i in range(row_number):
        for j in range(col_number):
            ws.cell(row=9+i,column=j+3).value = merge_pd.iat[i,j]

    wb.save('test.xlsx')
    print("数据已写入excel中!")

def plot_10(merge_pd):
    '''
    画前10大持仓股的曲线图
    '''
    temp_pd = merge_pd.sort_values(merge_pd.index[0],axis=1,ascendind=False)
    
    number = min(10,len(merge_pd))
    
    for i in range(number):
        pass


def save_to_config():

    first_dict = get_first_part()
    last_date = first_dict['last_date']

    if str(pd.Timestamp(last_date)) in close_data:
        print("该日期数据已经存在!")
        save_to_excel(config_dict['first_dict'],config_dict['second_dict'],pd.read_json(config_dict['old_pd'])) 
        return

    second_dict = get_second_part()
    third_pd = get_third_part(first_dict)
    merge_pd = third_pd.append(old_pd)
    merge_pd = merge_pd.sort_index(ascending=False)

    config_dict['name_dict'] = name_dict
    config_dict['close_data'] = close_data
    config_dict[last_date] = [first_dict,second_dict,third_pd.to_json()]
    config_dict['old_pd'] = merge_pd.to_json()
    config_dict['first_dict'] = first_dict
    config_dict['second_dict'] = second_dict
    config_dict['third_pd'] = third_pd.to_json()

    save_to_excel(first_dict,second_dict,merge_pd)
    
    with open('config.json','w',encoding='utf-8') as f:
        json.dump(config_dict,f)

    print('数据更新至%s!' % merge_pd.index[0])


In [32]:
    temp_html = soup.find("table",{"id":"Table5"})
    temp_html = temp_html.find_all('td',{"class":"arial12black"})

    first_dict = {}
    temp_string = temp_html[1].get_text().strip()
    temp_string = datetime.datetime.strptime(temp_string,"%d/%m/%Y")
    temp_string = temp_string.strftime("%Y-%m-%d")
    first_dict['last_date'] = temp_string
    first_dict['stock_code'] = temp_html[3].get_text().strip()
    first_dict['stock_name'] = temp_html[5].get_text().strip()

In [33]:
first_dict

{'last_date': '2018-06-20', 'stock_code': '91888', 'stock_name': '中國國旅'}

In [35]:
pd.DataFrame(first_dict,index=[])

,last_date,stock_code,stock_name


In [36]:
s1 = pd.DataFrame([1,2,3])

In [37]:
s1

,0
0,1
1,2
2,3


In [39]:
config_dict = {}

In [40]:
config_dict['test'] = s1

In [41]:
import json

In [42]:
json.dumps(config_dict)

TypeError: Object of type 'DataFrame' is not JSON serializable

In [30]:
temp_df = ts.get_k_data('601888')

In [31]:
temp_df

,date,open,close,high,low,volume,code
0,2015-11-09,27.577,28.112,28.300,27.577,106314.0,601888
1,2015-11-10,27.919,27.634,27.938,27.292,53714.0,601888
2,2015-11-11,27.630,27.331,27.716,26.940,69838.0,601888
3,2015-11-12,27.302,27.528,27.818,27.239,60645.0,601888
4,2015-11-13,27.268,26.535,27.475,26.400,56899.0,601888
5,2015-11-16,25.556,26.256,26.275,25.556,60722.0,601888
6,2015-11-17,26.347,26.039,26.636,25.990,89660.0,601888
7,2015-11-18,25.986,25.913,26.198,25.711,37370.0,601888
8,2015-11-19,25.957,26.323,26.371,25.798,29893.0,601888
9,2015-11-20,26.347,26.482,26.641,26.130,36738.0,601888


In [16]:
temp_html = soup.find("table",{"id":{"participantShareholdingList"}})
temp_html = temp_html.find_all('tr')

In [18]:
temp_html

[<tr style="vertical-align: top;">
 <td class="arial12black" nowrap="nowrap" onclick="Re_sort('ParticipantID')" style="cursor: pointer; width: 10%;">
 <img height="10" src="../../image/spacer.gif" width="10"/>參與者編號
                                         <img src="images/up.gif"/>
 </td>
 <td class="arial12black" nowrap="nowrap" onclick="Re_sort('ParticipantName')" style="cursor: pointer; width: 33%;">
                                         中央結算系統參與者名稱<br/>(*即願意披露的投資者戶口持有人)
                                         <img src="images/up.gif"/>
 </td>
 <td class="arial12black" nowrap="nowrap">
                                         地址
                                     </td>
 <td class="arial12black" nowrap="nowrap" onclick="Re_sort('Shareholding')" style="cursor: pointer; width: 15%; text-align: right;">
                                         持股量
                                         <img src="images/down.gif"/>
 </td>
 <td class="arial12black" style="text-align: right; width:

In [17]:
row_number = len(temp_html)
col_number = len(temp_html[0].find_all('td'))

In [19]:
    d = {}
    for i in range(col_number):
        d[i] = []

In [20]:
 for i in range(1,row_number):
    temp_tr = temp_html[i]
    temp_td = temp_tr.find_all('td')
    
    if len(temp_td) !=col_number:
        continue
        
    for j in range(col_number):
        temp_string = temp_td[j].get_text().strip()
        d[j].append(temp_string)

In [21]:
d

{0: ['C00019',
  'C00039',
  'C00010',
  'C00100',
  'B01161',
  'B01110',
  'C00093',
  'B01565',
  'B01654',
  'B01491',
  'B01274',
  'B01224',
  'B01130',
  'B01121',
  'B01323',
  'C00033',
  'B01590',
  'B01143',
  'B01610',
  'B01762',
  'B01905',
  'B01829',
  'B01138',
  'B01858',
  'B01890',
  'B01943',
  'B01842',
  'B01497',
  'B01355',
  'B01826',
  'B01451',
  'B01115',
  'B01727',
  'B01086',
  'B01284',
  'B01556',
  'B01929',
  'B01668',
  'B01773',
  'B01228',
  'B01866',
  'B01345',
  'B01971',
  'B01372',
  'B01825',
  'B01947',
  'C00036',
  'B01353',
  'B01695',
  'C00042',
  'B01818',
  'B01967'],
 1: ['香港上海匯豐銀行有限公司',
  '渣打銀行(香港)有限公司',
  '花旗銀行',
  'JPMORGAN CHASE BANK, NATIONAL ASSOCIATION',
  'UBS SECURITIES HONG KONG LTD',
  'J.P. MORGAN BROKING (HONG KONG) LTD',
  'BNP PARIBAS SECURITIES SERVICES',
  '國泰君安證券(香港)有限公司',
  '中國國際金融香港證券有限公司',
  'CREDIT SUISSE SECURITIES (HONG KONG) LTD',
  'MORGAN STANLEY HONG KONG SECURITIES LTD',
  'MLFE LTD',
  '中銀國際證券有限公司',
  '

In [22]:
last_pd = pd.DataFrame(d)

In [23]:
last_pd

,0,1,2,3,4
0,C00019,香港上海匯豐銀行有限公司,HSBC WEALTH BUSINESS SERVICES 8/F TOWER 2 & 3 ...,"58,891,639",3.01%
1,C00039,渣打銀行(香港)有限公司,SECURITIES SERVICES 15/F STANDARD CHARTERED TO...,"37,543,481",1.92%
2,C00010,花旗銀行,9/F CITI TOWER ONE BAY EAST 83 HOI BUN ROAD KW...,"28,092,781",1.43%
3,C00100,"JPMORGAN CHASE BANK, NATIONAL ASSOCIATION",48/F ONE ISLAND EAST 18 WESTLANDS ROAD QUARRY ...,"24,135,220",1.23%
4,B01161,UBS SECURITIES HONG KONG LTD,42/F ONE EXCHANGE SQUARE 8 CONNAUGHT PLACE CEN...,"11,384,139",0.58%
5,B01110,J.P. MORGAN BROKING (HONG KONG) LTD,48/F ONE ISLAND EAST 18 WESTLANDS ROAD ISLAND ...,"10,639,173",0.54%
6,C00093,BNP PARIBAS SECURITIES SERVICES,21/F PCCW TOWER TAIKOO PLACE 979 KING'S ROAD Q...,"9,281,884",0.47%
7,B01565,國泰君安證券(香港)有限公司,27/F GRAND MILLENNIUM PLAZA 181 QUEEN'S ROAD C...,"8,684,492",0.44%
8,B01654,中國國際金融香港證券有限公司,29/F ONE INTERNATIONAL FINANCE CENTRE 1 HARBOU...,"7,012,025",0.35%
9,B01491,CREDIT SUISSE SECURITIES (HONG KONG) LTD,LEVEL 88 INTERNATIONAL COMMERCE CENTRE 1 AUSTI...,"5,166,384",0.26%


In [25]:
    last_dict = {}
    for i in range(len(last_pd)):
        last_dict[last_pd.iat[i,0]] = last_pd.iat[i,1]

In [26]:
last_dict

{'B01086': '新鴻基投資服務有限公司',
 'B01110': 'J.P. MORGAN BROKING (HONG KONG) LTD',
 'B01115': '申萬宏源證券(香港)有限公司',
 'B01121': '法國興業證券(香港)有限公司',
 'B01130': '中銀國際證券有限公司',
 'B01138': '中信里昂證券有限公司',
 'B01143': '海通國際證券有限公司',
 'B01161': 'UBS SECURITIES HONG KONG LTD',
 'B01224': 'MLFE LTD',
 'B01228': '中信証券經紀(香港)有限公司',
 'B01274': 'MORGAN STANLEY HONG KONG SECURITIES LTD',
 'B01284': '恒生證券有限公司',
 'B01323': '德意志證券亞洲有限公司',
 'B01345': '輝立証券(香港)有限公司',
 'B01353': '大華繼顯(香港)有限公司',
 'B01355': '中國光大證券(香港)有限公司',
 'B01372': '第一金和昇證券有限公司',
 'B01451': '高盛(亞洲)證券有限公司',
 'B01491': 'CREDIT SUISSE SECURITIES (HONG KONG) LTD',
 'B01497': '永豐金證券(亞洲)有限公司',
 'B01556': '六福證券(香港)有限公司',
 'B01565': '國泰君安證券(香港)有限公司',
 'B01590': '盈透證券香港有限公司',
 'B01610': '凱基證券亞洲有限公司',
 'B01654': '中國國際金融香港證券有限公司',
 'B01668': '耀才證券國際(香港)有限公司',
 'B01695': '大新證券有限公司',
 'B01727': '工銀亞洲証券有限公司',
 'B01762': '星展唯高達香港有限公司',
 'B01773': '東洋証券亞洲有限公司',
 'B01818': '一通投資者有限公司',
 'B01825': '國元証券經紀(香港)有限公司',
 'B01826': '廣發証券(香港)經紀有限公司',
 'B01829': '華泰金融控股(香港)有限公司',


In [27]:
    new_pd = pd.DataFrame(last_pd[3])
    new_pd = new_pd.T
    new_pd.columns = last_pd[0]

In [29]:
print(new_pd)

0      C00019      C00039      C00010      C00100      B01161      B01110  \
3  58,891,639  37,543,481  28,092,781  24,135,220  11,384,139  10,639,173   

0     C00093     B01565     B01654     B01491  ...   B01971 B01372 B01825  \
3  9,281,884  8,684,492  7,012,025  5,166,384  ...    3,200  3,000  2,000   

0 B01947 C00036 B01353 B01695 C00042 B01818 B01967  
3  1,700  1,500  1,100  1,000  1,000    100    100  

[1 rows x 52 columns]


In [ ]:
    d = {}
    for i in range(col_number):
        d[i] = []

    for i in range(1,row_number):
        temp_tr = temp_html[i]
        temp_td = temp_tr.find_all('td')
    
        if len(temp_td) !=col_number:
            continue
        
        for j in range(col_number):
            temp_string = temp_td[j].get_text().strip()
            d[j].append(temp_string)

    last_pd = pd.DataFrame(d)
    
    last_dict = {}
    for i in range(len(last_pd)):
        last_dict[last_pd.iat[i,0]] = last_pd.iat[i,1]

    name_dict.update(last_dict)

    new_pd = pd.DataFrame(last_pd[3])
    new_pd = new_pd.T
    new_pd.columns = last_pd[0]

In [26]:
URL = r'http://www.hkexnews.hk/sdw/search/searchsdw_c.aspx'

In [27]:
s=requests.Session()

In [28]:
r=s.get(URL)

In [6]:
r

<Response [200]>

In [18]:
r.text

'\r\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\r\n<html xmlns="http://www.w3.org/1999/xhtml">\r\n<head><title>\r\n\t:: HKEX :: HKEXnews ::\r\n</title><link href="css/hkex_css.css" rel="stylesheet" type="text/css" />\r\n    <script type="text/javascript" src="/script/hkex_scripts.js"></script>\r\n    <script type="text/javascript" src="/script/hkex_common_c.js"></script>\r\n    <script type="text/javascript" src="/script/hkex_setting_c.js"></script>\r\n    <script type="text/javascript" src="js/search_form_var_chi.js?v=20150720"></script>\r\n    <script type="text/javascript" src="js/search_form.js?v=20150720"></script>\r\n    <script type="text/javascript">\r\n        function PopUpStockList() {\r\n            shareholdingDate = $("select[id$=\'ddlShareholdingYear\']").val() + $("select[id$=\'ddlShareholdingMonth\']").val() + $("select[id$=\'ddlShareholdingDay\']").val();\r\n            window.open(\'stockl

In [9]:
soup=BeautifulSoup(r.content,'html.parser')

In [10]:
VIEWSTATE=soup.find(id="__VIEWSTATE")['value']
print(VIEWSTATE)

Ckzs+VaFUEDBzZtKQjL8H1DdOd0DKk5HSQ32/tzS8gH+TO1W2S8tjyZf6hmXM7avS2T/u2jdQlJ5D4T/t0hkQaQlfrd8hQKfLvJKZTYFV9m/kkOQVXsunk3uLA6HnJSrYgJAaXQdxh6mR7NrKIXPi+WK3sSA2FiV36I//ZNpgbyoOW6Cd6NncSNgKzmpxEciVxxErw==


In [11]:
VIEWSTATEGENERATOR=soup.find(id="__VIEWSTATEGENERATOR")['value']
print(VIEWSTATEGENERATOR)

3B50BBBD


In [12]:
EVENTVALIDATION=soup.find(id="__EVENTVALIDATION")['value']
print(EVENTVALIDATION)

EDddx7fxnrdp/107CfB06ZyVcpdsQh9RV/LJjPEi4kbot5/97UrfaR99PeyxiM0tTpGLNDIJOMduS3rr1uXADZL/I2jcKp2Sj3Mw4wy1o0XeVqPB+Ph1C8fhhgd8YsPk4GZnMDLxjMhsp29QaVP7k4kgI427pwByQ4jjfdTmtMVNMDW4Ip435K0JLZTw0EauEs7LL627OdvXycUt/qQDkxj3drqss2VFubwySNJUsGfq7+nJmfikzGY2kyzMkC21ZtlCTUZBRtI/eHCl1WLrI/PPw3gKGf98ZuCRX2Wkg4FPsclYQhpCJEVKstDgaTzvDjy2RovMl2XKnR+nasAkeJUtCjx3AV4icTybEjerLsC5bsaRh+bR9UwQim4oQgqgV7Y4dOlrMrW0xCJjAmXOAEQ/V2+5ur8+kcQDLxkD3OiVCSLeasSxnqujO0cjNpe+IkKom0hI+C/j8mN5K4u/e+b5O/SI+a9yuHJfm6ONpNNPmP2aqiW3Dx0diudOvUfsRFC1dS7RTejDbrDPhCiHsIYGJF6CodoqNBjJjAZAoe8Tjsmif4eUn3c/VWKDUC9b8ZjYfjmWEn3TIyhb7Oiq8tKsZtlVhqWh716Rpi5o1uFyztFtd7lBF9DsHh1dv0kGxDbksZj3pI/hgWy2k80vpVilcqaawVmNZqR+PPA/UT97dUANMB6njkPxbwP8p2xRTyQDWZB1z9XAbgBBlPtmdp2RZUH37OWrC44938bj0dJyG3YyRQ2VsdZfGWuU9vgC4S11UlVlqh/+1woZytJmPOgUUd2La1v3fOAmgOTr1AHvZjIUesJVmQPtYpNW6wuIWPIh8ybAg1Ph560omCqiZVswSANAvcJKMEhO3hCUhcapl4Bq1Sh5kCv4p2YCtTuP9i5HRbn/kfLnBhE+bfila3hbMp2roKj8kk0uKeASpfeXbnZgvIbr713fEUnTj9LjcTZ2WLJ0dvLE/xgoLNaoOQyx1wyPyh61Yur4Dkih

In [13]:
login_data={
"__VIEWSTATE":VIEWSTATE,
"__VIEWSTATEGENERATOR":VIEWSTATEGENERATOR,
"__EVENTVALIDATION":EVENTVALIDATION,
"ddlShareholdingDay":'21',
"ddlShareholdingMonth":'06',
"ddlShareholdingYear":"2018",
'txtStockCode':'91888'
           }


In [14]:
r=s.post(URL, data=login_data)

In [16]:
print(r.url)

http://www.hkexnews.hk/sdw/search/searchsdw_c.aspx


In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

In [35]:
print(driver.find_element_by_id("content").text)

Here is some important text you want to retrieve!
A button to click!


In [31]:
driver.close()

In [4]:
browser = webdriver.Chrome()

In [13]:
browser.get("http://www.hkexnews.hk/sdw/search/searchsdw_c.aspx")

In [11]:
from selenium.webdriver.common.keys import Keys

In [15]:
element = browser.find_element_by_id("ddlShareholdingDay")

In [16]:
element.send_keys("15")

In [17]:
element

<selenium.webdriver.remote.webelement.WebElement (session="4927b9dee3910b452742e0e3db6445fd", element="0.857161784554014-1")>

In [18]:
element = browser.find_element_by_xpath("//select[@name='ddlShareholdingDay']")

In [20]:
all_options = element.find_elements_by_tag_name("option")

In [31]:
browser.find_element_by_id("btnSearch").click()

In [35]:
html_txt = browser.page_source

In [36]:
soup = BeautifulSoup(html_txt,'html.parser')

In [37]:
first_part = get_first_part()
print(first_part)

{'last_date': '2018-06-20', 'stock_code': '91888', 'stock_name': '中國國旅'}


In [40]:
browser.get("http://www.hkexnews.hk/sdw/search/searchsdw_c.aspx")

In [43]:
browser.back()

In [83]:
browser = webdriver.Chrome()
browser.get("http://www.hkexnews.hk/sdw/search/searchsdw_c.aspx")

In [6]:
browser.get("http://www.hkexnews.hk/sdw/search/searchsdw_c.aspx")

In [9]:
begin_date = '2018-06-01'
end_date = '2018-06-20'

In [88]:
i[8:10]

'20'

In [92]:
temp5 = temp4[-10:-1]

In [93]:
temp5

10031    2018-06-06
10032    2018-06-07
10033    2018-06-08
10036    2018-06-11
10037    2018-06-12
10038    2018-06-13
10039    2018-06-14
10040    2018-06-15
10044    2018-06-19
Name: calendarDate, dtype: object

In [8]:
temp_time = '2018-06-20'

In [15]:
for temp_time in trade_cal:
    
    select = Select(browser.find_element_by_name('ddlShareholdingMonth'))
    select.select_by_value(temp_time[5:7])

    select = Select(browser.find_element_by_name('ddlShareholdingDay'))
    select.select_by_value(temp_time[8:10])

    select = Select(browser.find_element_by_name('ddlShareholdingYear'))
    select.select_by_value(temp_time[0:4])

    element = browser.find_element_by_id("txtStockCode")
    element.send_keys("91888")

    browser.find_element_by_id("btnSearch").click()

    html_txt = browser.page_source

    soup = BeautifulSoup(html_txt,'html.parser')
    
    print(get_first_part())

    browser.back()

{'last_date': '2018-06-01', 'stock_code': '91888', 'stock_name': '中國國旅'}
{'last_date': '2018-06-04', 'stock_code': '91888', 'stock_name': '中國國旅'}
{'last_date': '2018-06-05', 'stock_code': '91888', 'stock_name': '中國國旅'}
{'last_date': '2018-06-06', 'stock_code': '91888', 'stock_name': '中國國旅'}
{'last_date': '2018-06-07', 'stock_code': '91888', 'stock_name': '中國國旅'}
{'last_date': '2018-06-08', 'stock_code': '91888', 'stock_name': '中國國旅'}
{'last_date': '2018-06-11', 'stock_code': '91888', 'stock_name': '中國國旅'}
{'last_date': '2018-06-12', 'stock_code': '91888', 'stock_name': '中國國旅'}
{'last_date': '2018-06-13', 'stock_code': '91888', 'stock_name': '中國國旅'}
{'last_date': '2018-06-14', 'stock_code': '91888', 'stock_name': '中國國旅'}
{'last_date': '2018-06-15', 'stock_code': '91888', 'stock_name': '中國國旅'}
{'last_date': '2018-06-19', 'stock_code': '91888', 'stock_name': '中國國旅'}
{'last_date': '2018-06-20', 'stock_code': '91888', 'stock_name': '中國國旅'}


In [11]:
import tushare as ts

In [12]:
temp = ts.trade_cal()

In [104]:
begin_date = '2018-01-01'
end_date = '2018-06-20'

In [98]:
s1 = '20180612'

In [13]:
    trade_cal = temp
    trade_cal = trade_cal[trade_cal['calendarDate'] >= begin_date]
    trade_cal = trade_cal[trade_cal['calendarDate'] <= end_date]
    trade_cal = trade_cal[trade_cal['isOpen'] == 1]
    trade_cal = trade_cal['calendarDate']

In [14]:
trade_cal

10026    2018-06-01
10029    2018-06-04
10030    2018-06-05
10031    2018-06-06
10032    2018-06-07
10033    2018-06-08
10036    2018-06-11
10037    2018-06-12
10038    2018-06-13
10039    2018-06-14
10040    2018-06-15
10044    2018-06-19
10045    2018-06-20
Name: calendarDate, dtype: object

In [101]:
s1[6:8]

'12'

In [61]:
temp2 = temp1[temp1['calendarDate'] <= end_date]

In [63]:
temp3 = temp2[temp2['isOpen'] == 1]

In [67]:
temp4=temp3['calendarDate']

In [75]:
time1 = temp4.iat[0]

In [76]:
import datetime

In [77]:
time2 = datetime.datetime.strptime(time1,'%Y-%m-%d')

In [82]:
for i in temp4:
    print(i)

2017-01-03
2017-01-04
2017-01-05
2017-01-06
2017-01-09
2017-01-10
2017-01-11
2017-01-12
2017-01-13
2017-01-16
2017-01-17
2017-01-18
2017-01-19
2017-01-20
2017-01-23
2017-01-24
2017-01-25
2017-01-26
2017-02-03
2017-02-06
2017-02-07
2017-02-08
2017-02-09
2017-02-10
2017-02-13
2017-02-14
2017-02-15
2017-02-16
2017-02-17
2017-02-20
2017-02-21
2017-02-22
2017-02-23
2017-02-24
2017-02-27
2017-02-28
2017-03-01
2017-03-02
2017-03-03
2017-03-06
2017-03-07
2017-03-08
2017-03-09
2017-03-10
2017-03-13
2017-03-14
2017-03-15
2017-03-16
2017-03-17
2017-03-20
2017-03-21
2017-03-22
2017-03-23
2017-03-24
2017-03-27
2017-03-28
2017-03-29
2017-03-30
2017-03-31
2017-04-05
2017-04-06
2017-04-07
2017-04-10
2017-04-11
2017-04-12
2017-04-13
2017-04-14
2017-04-17
2017-04-18
2017-04-19
2017-04-20
2017-04-21
2017-04-24
2017-04-25
2017-04-26
2017-04-27
2017-04-28
2017-05-02
2017-05-03
2017-05-04
2017-05-05
2017-05-08
2017-05-09
2017-05-10
2017-05-11
2017-05-12
2017-05-15
2017-05-16
2017-05-17
2017-05-18
2017-05-19